In [1]:
import pandas as pd
import numpy as np

In [23]:
training_data = pd.read_csv("training.csv")
training_data

,shop_id,date_block_num,item_id,item_category_id,item_cnt_day
0,0,0,32,40,6.0
1,0,0,33,37,3.0
2,0,0,35,40,1.0
3,0,0,43,40,1.0
4,0,0,51,57,2.0
...,...,...,...,...,...
1609119,59,33,22087,83,6.0
1609120,59,33,22088,83,2.0
1609121,59,33,22091,83,1.0
1609122,59,33,22100,42,1.0


In [24]:
# Need to add month and the year to each row, possibly this may imporve the training of the random forest
def add_month(row):
    date_block_num = int(row['date_block_num'])
    month = (date_block_num+1)%12
    return month

def add_year(row):
    date_block_num = int(row['date_block_num'])
    year = (date_block_num//12) + 2013
    return year
    
    

In [25]:
training_data['year'] = training_data.apply(add_year, axis=1)
training_data['month'] = training_data.apply(add_month, axis=1)
training_data

,shop_id,date_block_num,item_id,item_category_id,item_cnt_day,year,month
0,0,0,32,40,6.0,2013,1
1,0,0,33,37,3.0,2013,1
2,0,0,35,40,1.0,2013,1
3,0,0,43,40,1.0,2013,1
4,0,0,51,57,2.0,2013,1
...,...,...,...,...,...,...,...
1609119,59,33,22087,83,6.0,2015,10
1609120,59,33,22088,83,2.0,2015,10
1609121,59,33,22091,83,1.0,2015,10
1609122,59,33,22100,42,1.0,2015,10


In [26]:
training_data_l = training_data[['shop_id','date_block_num','year','month',
                                 'item_id','item_category_id','item_cnt_day']]
training_data_l

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_day
0,0,0,2013,1,32,40,6.0
1,0,0,2013,1,33,37,3.0
2,0,0,2013,1,35,40,1.0
3,0,0,2013,1,43,40,1.0
4,0,0,2013,1,51,57,2.0
...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.0
1609120,59,33,2015,10,22088,83,2.0
1609121,59,33,2015,10,22091,83,1.0
1609122,59,33,2015,10,22100,42,1.0


In [27]:
training_data_new = training_data_l.rename(columns={'item_cnt_day': 'item_cnt_mnth'})
training_data_new

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnth
0,0,0,2013,1,32,40,6.0
1,0,0,2013,1,33,37,3.0
2,0,0,2013,1,35,40,1.0
3,0,0,2013,1,43,40,1.0
4,0,0,2013,1,51,57,2.0
...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.0
1609120,59,33,2015,10,22088,83,2.0
1609121,59,33,2015,10,22091,83,1.0
1609122,59,33,2015,10,22100,42,1.0


In [28]:
training_data_new.to_csv(r'training_v2.csv', index=False)

Adding monthly mean item_cnt_mnth for ach shop_id item_id pairs

In [32]:
item_mnthly_mean = training_data_new.groupby(['shop_id', 'item_id'], as_index=False)[['item_cnt_mnth']].mean()
item_mnthly_mean = item_mnthly_mean.rename(columns={'item_cnt_mnth': 'item_cnt_mnthly_mean'})
item_mnthly_mean

,shop_id,item_id,item_cnt_mnthly_mean
0,0,30,31.000000
1,0,31,11.000000
2,0,32,8.000000
3,0,33,3.000000
4,0,35,7.500000
...,...,...,...
424119,59,22154,1.000000
424120,59,22155,1.000000
424121,59,22162,3.200000
424122,59,22164,1.500000


In [33]:
training_data_new = pd.merge(training_data_new, item_mnthly_mean, 
                             how='left', on=['shop_id', 'item_id'])
training_data_new

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnth,item_cnt_mnthly_mean
0,0,0,2013,1,32,40,6.0,8.000000
1,0,0,2013,1,33,37,3.0,3.000000
2,0,0,2013,1,35,40,1.0,7.500000
3,0,0,2013,1,43,40,1.0,1.000000
4,0,0,2013,1,51,57,2.0,2.500000
...,...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.0,6.424242
1609120,59,33,2015,10,22088,83,2.0,6.187500
1609121,59,33,2015,10,22091,83,1.0,2.294118
1609122,59,33,2015,10,22100,42,1.0,1.000000


In [34]:
training_data_new = training_data_new[['shop_id','date_block_num','year','month',
                                 'item_id','item_category_id','item_cnt_mnthly_mean', 
                                      'item_cnt_mnth']]

In [35]:
training_data_new

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_mnth
0,0,0,2013,1,32,40,8.000000,6.0
1,0,0,2013,1,33,37,3.000000,3.0
2,0,0,2013,1,35,40,7.500000,1.0
3,0,0,2013,1,43,40,1.000000,1.0
4,0,0,2013,1,51,57,2.500000,2.0
...,...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.424242,6.0
1609120,59,33,2015,10,22088,83,6.187500,2.0
1609121,59,33,2015,10,22091,83,2.294118,1.0
1609122,59,33,2015,10,22100,42,1.000000,1.0


In [36]:
training_data_new.to_csv(r'training_v3.csv', index=False)

# Adding prev 3 monthly avg

In [2]:
train_data = pd.read_csv('training_v3.csv')
train_data

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_mnth
0,0,0,2013,1,32,40,8.000000,6.0
1,0,0,2013,1,33,37,3.000000,3.0
2,0,0,2013,1,35,40,7.500000,1.0
3,0,0,2013,1,43,40,1.000000,1.0
4,0,0,2013,1,51,57,2.500000,2.0
...,...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.424242,6.0
1609120,59,33,2015,10,22088,83,6.187500,2.0
1609121,59,33,2015,10,22091,83,2.294118,1.0
1609122,59,33,2015,10,22100,42,1.000000,1.0


In [3]:
test_val = train_data.groupby(['shop_id','item_id','date_block_num'])    

In [5]:
test_val.get_group((0,32,33))

KeyError: (0, 32, 33)

In [19]:
# Determine the average value of previous three months
def add_prev_monthly_mean(row, training_grouped):
    shop_id = row['shop_id']
    item_id = row['item_id']
    date_block_num = int(row['date_block_num'])
    key = (shop_id, item_id, date_block_num)
    total = 0
    for block_num in range(date_block_num-3, date_block_num):
        try:
            val = training_grouped.get_group((shop_id, item_id, block_num))
            total += int(val['item_cnt_mnth'])
        except KeyError as e:
            total += 0
    return float(total)/3.0

In [20]:
train_data['item_cnt_prev_3_mnth_mean'] = train_data.apply(add_prev_monthly_mean, args=(test_val,),axis=1)

In [21]:
train_data

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_mnth,item_cnt_prev_3_mnth_mean
0,0,0,2013,1,32,40,8.000000,6.0,0.000000
1,0,0,2013,1,33,37,3.000000,3.0,0.000000
2,0,0,2013,1,35,40,7.500000,1.0,0.000000
3,0,0,2013,1,43,40,1.000000,1.0,0.000000
4,0,0,2013,1,51,57,2.500000,2.0,0.000000
...,...,...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.424242,6.0,3.333333
1609120,59,33,2015,10,22088,83,6.187500,2.0,5.000000
1609121,59,33,2015,10,22091,83,2.294118,1.0,1.000000
1609122,59,33,2015,10,22100,42,1.000000,1.0,0.333333


In [12]:
test_val.get_group((59,22087,33))

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_mnth,item_cnt_prev_3_mnth_mean
1609119,59,33,2015,10,22087,83,6.424242,6.0,[nan]


In [13]:
test_val.get_group((59,22087,30))

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_mnth,item_cnt_prev_3_mnth_mean
1607536,59,30,2015,7,22087,83,6.424242,5.0,[nan]


In [14]:
test_val.get_group((59,22087,31))

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_mnth,item_cnt_prev_3_mnth_mean
1608127,59,31,2015,8,22087,83,6.424242,2.0,[nan]


In [15]:
test_val.get_group((59,22087,32))

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_mnth,item_cnt_prev_3_mnth_mean
1608619,59,32,2015,9,22087,83,6.424242,3.0,[nan]


In [16]:
val = test_val.get_group((59,22087,33))

In [25]:
train_data = train_data[['shop_id','date_block_num','year','month','item_id',
                         'item_category_id','item_cnt_mnthly_mean','item_cnt_prev_3_mnth_mean','item_cnt_mnth']]

In [26]:
train_data

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_prev_3_mnth_mean,item_cnt_mnth
0,0,0,2013,1,32,40,8.000000,0.000000,6.0
1,0,0,2013,1,33,37,3.000000,0.000000,3.0
2,0,0,2013,1,35,40,7.500000,0.000000,1.0
3,0,0,2013,1,43,40,1.000000,0.000000,1.0
4,0,0,2013,1,51,57,2.500000,0.000000,2.0
...,...,...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.424242,3.333333,6.0
1609120,59,33,2015,10,22088,83,6.187500,5.000000,2.0
1609121,59,33,2015,10,22091,83,2.294118,1.000000,1.0
1609122,59,33,2015,10,22100,42,1.000000,0.333333,1.0


In [27]:
train_data.to_csv('training_v4.csv', index=False)

# Adding price information

In [28]:
sales_data = pd.read_csv('sales_train.csv')
sales_data

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [52]:
sales_prices = sales_data.groupby(['shop_id','item_id','date_block_num'])['item_price'].min()

In [53]:
sales_prices

shop_id  item_id  date_block_num
0        30       1                 265.0
         31       1                 434.0
         32       0                 221.0
                  1                 221.0
         33       0                 347.0
                                    ...  
59       22164    27                699.0
                  30                699.0
         22167    9                 299.0
                  11                299.0
                  17                299.0
Name: item_price, Length: 1609124, dtype: float64

In [31]:
sales_grouped = sales_data.groupby(['shop_id','item_id','date_block_num'])

In [64]:
sales_grouped.get_group((59,22164,27))['item_price'].min()

699.0

In [54]:
sales_prices_reset = sales_prices.reset_index()

In [55]:
sales_prices_reset

,shop_id,item_id,date_block_num,item_price
0,0,30,1,265.0
1,0,31,1,434.0
2,0,32,0,221.0
3,0,32,1,221.0
4,0,33,0,347.0
...,...,...,...,...
1609119,59,22164,27,699.0
1609120,59,22164,30,699.0
1609121,59,22167,9,299.0
1609122,59,22167,11,299.0


In [65]:
# Determine the avg price from previous 3 months
def add_3mnth_avg_price(row, sales_grouped):
    shop_id = row['shop_id']
    item_id = row['item_id']
    date_block_num = int(row['date_block_num'])
    key = (shop_id, item_id, date_block_num)
    total = 0.0
    for block_num in range(date_block_num-3, date_block_num):
        try:
            val = sales_grouped.get_group((shop_id, item_id, block_num))
            total += float(val['item_price'].min())
        except KeyError as e:
            total += 0
    return float(total)/3.0

In [66]:
train_data['avg_item_price_3mnth_ago'] = train_data.apply(add_3mnth_avg_price,axis=1, args=(sales_grouped,))

/Users/shreyash/.virtualenvs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [67]:
train_data

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_prev_3_mnth_mean,item_cnt_mnth,avg_item_price_3mnth_ago
0,0,0,2013,1,32,40,8.000000,0.000000,6.0,0.000000
1,0,0,2013,1,33,37,3.000000,0.000000,3.0,0.000000
2,0,0,2013,1,35,40,7.500000,0.000000,1.0,0.000000
3,0,0,2013,1,43,40,1.000000,0.000000,1.0,0.000000
4,0,0,2013,1,51,57,2.500000,0.000000,2.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.424242,3.333333,6.0,119.000000
1609120,59,33,2015,10,22088,83,6.187500,5.000000,2.0,119.000000
1609121,59,33,2015,10,22091,83,2.294118,1.000000,1.0,59.666667
1609122,59,33,2015,10,22100,42,1.000000,0.333333,1.0,209.666667


In [69]:
sales_grouped.get_group((59,22091,33))

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2899136,03.10.2015,33,59,22091,179.0,1.0


In [72]:
sales_grouped.get_group((59,22091,32))

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2845853,10.09.2015,32,59,22091,179.0,1.0
2845854,17.09.2015,32,59,22091,179.0,1.0
2845855,22.09.2015,32,59,22091,179.0,1.0


In [74]:
train_data = train_data[['shop_id','date_block_num','year','month','item_id',
                         'item_category_id','item_cnt_mnthly_mean','item_cnt_prev_3_mnth_mean',
                         'avg_item_price_3mnth_ago','item_cnt_mnth']]

In [75]:
train_data

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_prev_3_mnth_mean,avg_item_price_3mnth_ago,item_cnt_mnth
0,0,0,2013,1,32,40,8.000000,0.000000,0.000000,6.0
1,0,0,2013,1,33,37,3.000000,0.000000,0.000000,3.0
2,0,0,2013,1,35,40,7.500000,0.000000,0.000000,1.0
3,0,0,2013,1,43,40,1.000000,0.000000,0.000000,1.0
4,0,0,2013,1,51,57,2.500000,0.000000,0.000000,2.0
...,...,...,...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.424242,3.333333,119.000000,6.0
1609120,59,33,2015,10,22088,83,6.187500,5.000000,119.000000,2.0
1609121,59,33,2015,10,22091,83,2.294118,1.000000,59.666667,1.0
1609122,59,33,2015,10,22100,42,1.000000,0.333333,209.666667,1.0


In [76]:
train_data.to_csv('training_v5.csv', index=False)

# Adding a range of previously montly sales

In [77]:
def get_prev_n_monthly_sales_avg(row, n, training_grouped):
    shop_id = row['shop_id']
    item_id = row['item_id']
    date_block_num = int(row['date_block_num'])
    key = (shop_id, item_id, date_block_num)
    total = 0
    for block_num in range(date_block_num-n, date_block_num):
        try:
            val = training_grouped.get_group((shop_id, item_id, block_num))
            total += int(val['item_cnt_mnth'])
        except KeyError as e:
            total += 0
    return float(total)/n

In [79]:
train_data['item_cnt_prev_4_mnth_mean'] = train_data.apply(get_prev_n_monthly_sales_avg,
                                                          args=(4, test_val,), axis=1)

/Users/shreyash/.virtualenvs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [80]:
train_data

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_prev_3_mnth_mean,avg_item_price_3mnth_ago,item_cnt_mnth,item_cnt_prev_4_mnth_mean
0,0,0,2013,1,32,40,8.000000,0.000000,0.000000,6.0,0.00
1,0,0,2013,1,33,37,3.000000,0.000000,0.000000,3.0,0.00
2,0,0,2013,1,35,40,7.500000,0.000000,0.000000,1.0,0.00
3,0,0,2013,1,43,40,1.000000,0.000000,0.000000,1.0,0.00
4,0,0,2013,1,51,57,2.500000,0.000000,0.000000,2.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.424242,3.333333,119.000000,6.0,2.75
1609120,59,33,2015,10,22088,83,6.187500,5.000000,119.000000,2.0,4.75
1609121,59,33,2015,10,22091,83,2.294118,1.000000,59.666667,1.0,0.75
1609122,59,33,2015,10,22100,42,1.000000,0.333333,209.666667,1.0,0.25


In [81]:
train_data = train_data[['shop_id','date_block_num','year','month','item_id',
                         'item_category_id','item_cnt_mnthly_mean','item_cnt_prev_3_mnth_mean',
                         'item_cnt_prev_4_mnth_mean','avg_item_price_3mnth_ago','item_cnt_mnth']]

In [82]:
train_data

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_prev_3_mnth_mean,item_cnt_prev_4_mnth_mean,avg_item_price_3mnth_ago,item_cnt_mnth
0,0,0,2013,1,32,40,8.000000,0.000000,0.00,0.000000,6.0
1,0,0,2013,1,33,37,3.000000,0.000000,0.00,0.000000,3.0
2,0,0,2013,1,35,40,7.500000,0.000000,0.00,0.000000,1.0
3,0,0,2013,1,43,40,1.000000,0.000000,0.00,0.000000,1.0
4,0,0,2013,1,51,57,2.500000,0.000000,0.00,0.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.424242,3.333333,2.75,119.000000,6.0
1609120,59,33,2015,10,22088,83,6.187500,5.000000,4.75,119.000000,2.0
1609121,59,33,2015,10,22091,83,2.294118,1.000000,0.75,59.666667,1.0
1609122,59,33,2015,10,22100,42,1.000000,0.333333,0.25,209.666667,1.0


In [84]:
train_data.to_csv('training_v6.csv', index=False)